# Getting Started with Propnet

The Propnet object tells us all about the property types and models Propnet understands

In [1]:
from propnet import Propnet

Test value for p_wave_modulus is zero. Please change to a more appropriate test value.
Test value for lame_first_parameter is zero. Please change to a more appropriate test value.


In [2]:
p = Propnet()

You can print Propnet to see the property types and models it supports.

In [3]:
print(p)

Propnet Graph

Property Types:
	 Elastic tensor (in Voigt notation)
	 P-wave modulus
	 Young's modulus
	 Poisson ratio
	 Lamé's first parameter
	 Compliance tensor (in Voigt notation)
	 Shear modulus
	 Bulk modulus
Models:
	 Convert between elastic tensors and compliance tensors (in Voigt notation)
	 Convert between isotropic elastic moduli
	 Calculate isotropic elastic moduli from elastic tensor


Or you can iterate over the graph to see how it works behind the scenes.

In [4]:
for n in p.graph.nodes():
    print(n)

PropertyType.elastic_tensor_voigt
PropertyType.p_wave_modulus
PropertyType.youngs_modulus
PropertyType.poisson_ratio
PropertyType.lame_first_parameter
PropertyType.compliance_tensor_voigt
PropertyType.shear_modulus
PropertyType.bulk_modulus
<class 'propnet.models.ElasticComplianceVoigtConverter.ElasticComplianceVoigtConverter'>
<class 'propnet.models.IsotropicElasticModuli.IsotropicElasticModuli'>
<class 'propnet.models.IsotropicElasticModuliFromTensor.IsotropicElasticModuliFromTensor'>


# Define a Material

In [5]:
from propnet import Material

In [6]:
m = Material()

You can define the material's formula, or leave it blank for an anonymous material.

In [7]:
m.add_formula('Si')

You can also add a property to the material.

In [8]:
m.add_property('poisson_ratio', 0.2)

TypeError: __init__() missing 1 required positional argument: 'comment'

# Add the Material to Propnet and Evaluate

In [9]:
p.add_material(m)

You can now evaluate Propnet and retrieve all properties for the material that it knows how to calculate, and this is returned as a Pandas DataFrame.

In [10]:
# TODO: p.evaluate()

# Running a Model Manually

If you just want to run a single model and already know your inputs, this is easy to do!

In [11]:
from propnet.models.IsotropicElasticModuli import IsotropicElasticModuli

In [12]:
model = IsotropicElasticModuli()

You can see the symbols it supports:

In [13]:
model.symbol_mapping

{'E': 'youngs_modulus', 'G': 'shear_modulus', 'n': 'poisson_ratio'}

And can evaluate the model:

In [14]:
model.evaluate({'E': 120, 'n': 0.5}, symbol_out='G')

Units are not defined for your E, using assumed units from property definition.
Units are not defined for your n, using assumed units from property definition.


KeyError: 'gigapascal'

In [15]:
from pint import UnitRegistry

In [16]:
ureg = UnitRegistry()

In [19]:
ureg.parse_expression("gigapascal").to_tuple()

(1, (('gigapascal', 1.0),))

In [24]:
node_list = list(p.graph.nodes)

In [27]:
from enum import Enum

In [32]:
idx = 10
print(node_list[idx])
print(type(node_list[idx]))
if isinstance(node_list[idx], Enum):
    print(node_list[idx].value)
    print(type(node_list[idx].value))

<class 'propnet.models.ElasticComplianceVoigtConverter.ElasticComplianceVoigtConverter'>
<class 'abc.ABCMeta'>


In [33]:
my_material = Material.from_mpid("mp-12345")
p.add_material(my_material)

AttributeError: type object 'Material' has no attribute 'from_mpid'

# Defining a Model with Constraints

In [ ]:
class MySampleModel(AbstractModel):
    
    